# Machine Learning - Precios Propiedades (Versión 3)

In [ ]:
import pandas as pd

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics

import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 0. Lectura de datos

In [ ]:
DIR = "/content/drive/MyDrive/DATA MINING/fcen-dm-2025-prediccion-precio-de-propiedades/entrenamiento"

In [ ]:
df_ent = pd.read_csv(f"{DIR}/entrenamiento.csv", index_col="id")
df_ent.head()

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,currency,price_period,title,description,property_type,operation_type,price
id,,,,,,,,,,,,,,,,,,,,,
556713,Propiedad,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Argentina,Capital Federal,Colegiales,NaN,...,2.0,NaN,NaN,USD,NaN,"Departamento en Venta en Belgrano, Capital fed...","Sup total por escritura: 96,47 m2 (cubiertos: ...",Departamento,Venta,259000.0
257134,Propiedad,2019-08-17,9999-12-31,2019-08-17,-58.420737,-34.631770,Argentina,Capital Federal,Boedo,NaN,...,1.0,74.0,47.0,USD,NaN,PH Venta Boedo 2 amb Patio,Corredor Responsable: MARCELO TRUJILLO - CPI ...,PH,Venta,140000.0
521738,Propiedad,2019-08-05,2019-08-31,2019-08-05,-58.429983,-34.607225,Argentina,Capital Federal,Almagro,NaN,...,1.0,66.0,64.0,USD,NaN,Venta 3 Ambientes - Almagro - Balcón - Ameniti...,Corredor Responsable: Marcelo Trujillo - CUCIC...,Departamento,Venta,173000.0
383947,Propiedad,2019-10-23,2019-11-08,2019-10-23,-58.420780,-34.608640,Argentina,Capital Federal,Almagro,NaN,...,1.0,55.0,55.0,USD,NaN,Depto.tipo casa de 3 ambientes en Venta en Alm...,"Muy Lindo ph al contrafrente con 2 patios!, 80...",PH,Venta,155000.0
986212,Propiedad,2019-08-01,2019-08-14,2019-08-01,-58.371394,-34.621835,Argentina,Capital Federal,San Telmo,NaN,...,1.0,42.0,42.0,USD,NaN,Ph en edificio de estilo Frances . 2 dormitori...,PH en Edificio Frances en Estado original!!!al...,PH,Venta,74000.0


In [ ]:
# cantidad de filas y columnas
df_ent.shape

(985180, 24)

## 1. Entender los datos (AID)

## 2. Limpiar y transformar los datos (MD)

In [ ]:
# Selección de datos.
df_ent = df_ent.loc[(df_ent["price"].notna())  & (df_ent["currency"].isin(["USD", "ARS"])) & \
    (df_ent["l2"] == "Capital Federal") & (df_ent["operation_type"] == 'Venta') & \
    (df_ent["property_type"].isin(["Cochera", "Departamento", "Casa"]))]
df_ent.shape

(104839, 24)

In [ ]:
df_ent.loc[df_ent["l3"] == "Distrito Audiovisual", "l3"] = "Chacarita"
df_ent.loc[df_ent["l3"] == "Catalinas", "l3"] = "Retiro"
df_ent.loc[:, "l4"] = df_ent["l4"].fillna(df_ent["l3"])

In [ ]:
#PRECIO POR METRO CUADRADO <500 O >10000 (PARA PROPIEDADES ATÍPICAS Y OUTLIERS)
df_ent["price_m2"] = df_ent["price"] / (df_ent["surface_total"] + 0.001)
df_ent = df_ent[(df_ent["price_m2"] >= 500) & (df_ent["price_m2"] <= 10000)]

<ipython-input-58-c986b4658799>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ent["price_m2"] = df_ent["price"] / (df_ent["surface_total"] + 0.001)


In [ ]:
PESOS_POR_DOLAR = 78.5
df_ent.loc[df_ent["currency"] == "ARS", "price"] = df_ent.loc[df_ent["currency"] == "ARS", "price"] / PESOS_POR_DOLAR

In [ ]:
df_ent.loc[:, "bedrooms"] = df_ent["bedrooms"].fillna(df_ent["rooms"] - 1)
df_ent.loc[:, "rooms"] = df_ent["rooms"].fillna(df_ent["bedrooms"] + 1)


In [ ]:
#BATHS
df_ent.loc[:, "bathrooms"] = df_ent["bathrooms"].fillna(df_ent["rooms"] - df_ent["bedrooms"])

In [ ]:
df_ent = df_ent.dropna(subset=["surface_total"])

In [ ]:
df_ent.loc[df_ent['property_type'] == 'Cochera', ['bathrooms', 'bedrooms', 'rooms']] = 0

In [ ]:
df_ent["surface_total"] = df_ent["surface_total"].fillna(df_ent["surface_covered"])
df_ent = df_ent.loc[df_ent["surface_total"].notna()]

In [ ]:
df_ent.loc[:, "l4"] = df_ent["l4"].fillna(df_ent["l3"])

In [ ]:
df_ent["price_m2"] = df_ent["price"] / (df_ent["surface_total"] + 0.001)
gb = df_ent.groupby(by=["property_type", "l4"])["price_m2"].mean().reset_index()
gb

,property_type,l4,price_m2
0,Casa,Abasto,1638.860004
1,Casa,Agronomía,1639.703726
2,Casa,Almagro,1160.869491
3,Casa,Balvanera,1563.899913
4,Casa,Barracas,1249.043058
...,...,...,...
164,Departamento,Villa Riachuelo,1551.667721
165,Departamento,Villa Santa Rita,2182.146642
166,Departamento,Villa Soldati,863.765979
167,Departamento,Villa Urquiza,2832.556270


In [ ]:
df_ent = df_ent[df_ent.columns.drop("price_m2")].merge(gb, on=["property_type", "l4"], how="left")
df_ent["price_m2_x_surface_total"] = df_ent["price_m2"] * df_ent["surface_total"]

In [ ]:
#LIMPIAR PROPIEDADES QUE SE DESVIAN DEL PRECIO POR METRO CUADRADO (USD 1000)
p = (df_ent["price"] / (df_ent["surface_total"] + 0.001) - df_ent["price_m2"]) > 1000
df_ent = df_ent[~p]

In [ ]:
df_ent["ratio_cubierta"] = df_ent["surface_covered"] / df_ent["surface_total"]
df_ent["diferencia_superficie"] = df_ent["surface_total"] - df_ent["surface_covered"]

In [ ]:
df_ent = df_ent.drop_duplicates()

In [ ]:
df_ent = df_ent[~((df_ent['property_type'] == 'Cochera') & (df_ent['surface_total'] > 80))]

In [ ]:
df_ent = pd.get_dummies(df_ent, columns=["property_type", "l4"], prefix=["prop", "l4"])

In [ ]:
df_ent['rooms_to_surface_total'] = np.where(df_ent['surface_total'] != 0, df_ent['rooms'] / df_ent['surface_total'], np.nan)
df_ent['bedrooms_to_rooms'] = np.where(df_ent['rooms'] != 0, df_ent['bedrooms'] / df_ent['rooms'], np.nan)
df_ent['surface_covered_to_surface_total'] = np.where(df_ent['surface_total'] != 0, df_ent['surface_covered'] / df_ent['surface_total'], np.nan)
df_ent['bathrooms_to_rooms'] = np.where(df_ent['rooms'] != 0, df_ent['bathrooms'] / df_ent['rooms'], np.nan)
df_ent['bathrooms_to_bedrooms'] = np.where(df_ent['bedrooms'] != 0, df_ent['bathrooms'] / df_ent['bedrooms'], np.nan)
df_ent['rooms_plus_bedrooms'] = df_ent['rooms'] + df_ent['bedrooms']
df_ent['rooms_minus_bedrooms'] = df_ent['rooms'] - df_ent['bedrooms']

In [ ]:
df_ent["a_estrenar"] = df_ent["description"].str.lower().str.contains("estrenar", na=False)

In [ ]:
df_ent= df_ent.drop([
    "ad_type", "l6", "title", "description", "start_date", "end_date", "created_on",
    "currency", "price_period", "operation_type", "l1", "l2", "l5"
], axis=1)

In [ ]:
df_ent.head()


,lat,lon,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,price_m2,...,l4_Villa Urquiza,l4_Villa del Parque,rooms_to_surface_total,bedrooms_to_rooms,surface_covered_to_surface_total,bathrooms_to_rooms,bathrooms_to_bedrooms,rooms_plus_bedrooms,rooms_minus_bedrooms,a_estrenar
0,-58.429983,-34.607225,Almagro,3.0,2.0,1.0,66.0,64.0,173000.0,2514.532457,...,False,False,0.045455,0.666667,0.969697,0.333333,0.5,5.0,1.0,False
1,-58.404054,-34.599623,Barrio Norte,3.0,2.0,2.0,91.0,86.0,190000.0,3107.281544,...,False,False,0.032967,0.666667,0.945055,0.666667,1.0,5.0,1.0,False
2,-58.421650,-34.588602,Palermo,3.0,2.0,2.0,80.0,80.0,240000.0,3445.971902,...,False,False,0.037500,0.666667,1.000000,0.666667,1.0,5.0,1.0,False
3,-58.378095,-34.608478,Monserrat,2.0,1.0,1.0,48.0,43.0,99000.0,2071.624548,...,False,False,0.041667,0.500000,0.895833,0.500000,1.0,3.0,1.0,False
4,-58.420295,-34.606271,Almagro,3.0,2.0,2.0,80.0,76.0,190000.0,2514.532457,...,False,False,0.037500,0.666667,0.950000,0.666667,1.0,5.0,1.0,False


In [ ]:
df_ent = df_ent.fillna(0)

## 3. Entrenamiento del modelos (AA)

In [ ]:
# Me quedo con las columnas numéricas o booleanas solamente
df_ent = df_ent.select_dtypes(['number', 'bool'])
print(df_ent.columns)
X = df_ent[df_ent.columns.drop('price')]
y = df_ent['price']

Index(['lat', 'lon', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'price', 'price_m2', 'price_m2_x_surface_total',
       'ratio_cubierta', 'diferencia_superficie', 'prop_Casa', 'prop_Cochera',
       'prop_Departamento', 'l4_Abasto', 'l4_Agronomía', 'l4_Almagro',
       'l4_Balvanera', 'l4_Barracas', 'l4_Barrio Norte', 'l4_Belgrano',
       'l4_Boca', 'l4_Boedo', 'l4_Caballito', 'l4_Centro / Microcentro',
       'l4_Chacarita', 'l4_Coghlan', 'l4_Colegiales', 'l4_Congreso',
       'l4_Constitución', 'l4_Flores', 'l4_Floresta', 'l4_Las Cañitas',
       'l4_Liniers', 'l4_Mataderos', 'l4_Monserrat', 'l4_Monte Castro',
       'l4_Nuñez', 'l4_Once', 'l4_Palermo', 'l4_Palermo Chico',
       'l4_Palermo Hollywood', 'l4_Palermo Soho', 'l4_Palermo Viejo',
       'l4_Parque Avellaneda', 'l4_Parque Centenario', 'l4_Parque Chacabuco',
       'l4_Parque Chas', 'l4_Parque Patricios', 'l4_Paternal', 'l4_Pompeya',
       'l4_Puerto Madero', 'l4_Recoleta', 'l4_Retiro', '

In [ ]:
### Versión con train/test split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

best_score_test = 1000000


for n_estimators in [10, 50, 100, 300, 500, 700, 1000]:

    # for max_depth in [3, 5, 8, 10, 12]:
    # 	reg = sk.ensemble.GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    for max_depth in [5, 10, 20, 30, 50, None]:
        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=10, min_samples_split=20, n_jobs=-1, random_state=42)

        # Entrenamos el modelo
        _ = reg.fit(X_train, y_train)

        # Cálculo del error en entrenamiento (train)
        y_pred = reg.predict(X_train)
        score_train = sk.metrics.root_mean_squared_error(y_train, y_pred)

        # Cálculo del error en prueba (test)
        y_pred = reg.predict(X_test)
        score_test  = sk.metrics.root_mean_squared_error(y_test,  y_pred)

        best_score_test = min(best_score_test, score_test)

        print(f"{n_estimators=} -- {max_depth=} --> {score_train=:.2f} - {score_test=:.2f} - {best_score_test=:.2f}")

n_estimators=10 -- max_depth=5 --> score_train=77204.10 - score_test=73450.55 - best_score_test=73450.55
n_estimators=10 -- max_depth=10 --> score_train=63899.03 - score_test=64553.38 - best_score_test=64553.38
n_estimators=10 -- max_depth=20 --> score_train=58969.67 - score_test=62543.78 - best_score_test=62543.78
n_estimators=10 -- max_depth=30 --> score_train=58866.87 - score_test=62312.15 - best_score_test=62312.15
n_estimators=10 -- max_depth=50 --> score_train=58908.10 - score_test=62439.11 - best_score_test=62312.15
n_estimators=10 -- max_depth=None --> score_train=58908.10 - score_test=62439.11 - best_score_test=62312.15
n_estimators=50 -- max_depth=5 --> score_train=76973.16 - score_test=73215.93 - best_score_test=62312.15
n_estimators=50 -- max_depth=10 --> score_train=63021.87 - score_test=63801.10 - best_score_test=62312.15
n_estimators=50 -- max_depth=20 --> score_train=57937.04 - score_test=61561.46 - best_score_test=61561.46
n_estimators=50 -- max_depth=30 --> score_trai

KeyboardInterrupt: 

In [ ]:
### Versión con validación cruzada

best_score = 1000000

### Usar sólo RandomForestRegressor o GradientBoostingRegressor
for n_estimators in [10, 50, 100, 300, 500, 700, 1000]:

    # for max_depth in [3, 5, 8, 10, 12]:
    # 	reg = sk.ensemble.GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)

    for max_depth in [5, 10, 20, 30, 50, None]:
        reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=10, min_samples_split=20, n_jobs=-1, random_state=42)

        # Entrenamos el modelo (cross validation en 5 partes)
        scores = -1*sk.model_selection.cross_val_score(reg, X, y, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)

        scores_mean = scores.mean()
        scores_std = scores.std()

        best_score = min(best_score, scores_mean)

        print(f"{n_estimators=} -- {max_depth=} --> {scores_mean=:.2f} - {scores_std=:.2f} - {best_score=:.2f}")

n_estimators=10 -- max_depth=5 --> scores_mean=80877.68 - scores_std=15921.12 - best_score=80877.68
n_estimators=10 -- max_depth=10 --> scores_mean=72676.79 - scores_std=17143.81 - best_score=72676.79
n_estimators=10 -- max_depth=20 --> scores_mean=70656.81 - scores_std=17427.47 - best_score=70656.81
n_estimators=10 -- max_depth=30 --> scores_mean=70602.27 - scores_std=17472.71 - best_score=70602.27
n_estimators=10 -- max_depth=50 --> scores_mean=70606.29 - scores_std=17470.94 - best_score=70602.27
n_estimators=10 -- max_depth=None --> scores_mean=70606.29 - scores_std=17470.94 - best_score=70602.27
n_estimators=50 -- max_depth=5 --> scores_mean=80049.01 - scores_std=15056.16 - best_score=70602.27
n_estimators=50 -- max_depth=10 --> scores_mean=71455.23 - scores_std=16266.95 - best_score=70602.27


KeyboardInterrupt: 

## 4. Dataframe A Predecir



In [ ]:
df_ap = pd.read_csv(f"/content/drive/MyDrive/DATA MINING/fcen-dm-2025-prediccion-precio-de-propiedades/a_predecir.csv", index_col="id")
df_ap.head()

,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,...,bathrooms,surface_total,surface_covered,currency,price_period,title,description,property_type,operation_type,price
id,,,,,,,,,,,,,,,,,,,,,
1068,Propiedad,2020-06-21,2020-06-27,2020-06-21,NaN,NaN,Argentina,Capital Federal,Palermo,Palermo Viejo,...,1.0,286.0,160.0,USD,NaN,Casa de 4 ambientes en Venta en Palermo viejo,TRIPLEX EN MEDIO TERRENO DE 4 AMB MUY AMPLIOS...,Casa,Venta,NaN
1069,Propiedad,2020-06-21,2020-06-27,2020-06-21,-58.426576,-34.590987,Argentina,Capital Federal,Palermo,NaN,...,5.0,444.0,270.0,USD,NaN,Casa de 8 ambientes en Venta en Palermo,OPORTUNIDAD!! Edificio en Block desarrollado e...,Casa,Venta,NaN
1073,Propiedad,2020-06-21,2020-06-24,2020-06-21,-58.479808,-34.631266,Argentina,Capital Federal,Floresta,NaN,...,2.0,166.0,120.0,USD,NaN,Venta de casa en PH en Floresta,Casa con muy buena distribución y de amplios a...,Casa,Venta,NaN
1082,Propiedad,2020-06-21,2020-06-27,2020-06-21,-58.437889,-34.603291,Argentina,Capital Federal,Villa Crespo,NaN,...,3.0,352.0,185.0,USD,NaN,Casa de 5 ambientes en Venta en Villa crespo,SON 185 METROS CUBIERNTOS CON COCHERA MAS PATI...,Casa,Venta,NaN
1091,Propiedad,2020-06-21,2020-06-27,2020-06-21,-58.476461,-34.602494,Argentina,Capital Federal,Villa del Parque,NaN,...,2.0,168.0,106.0,USD,NaN,Venta de casa de tres ambientes en Villa del P...,"Hermosa casa ubicada en zona residencial, sobr...",Casa,Venta,NaN


In [ ]:
df_ap.shape

(7012, 24)

In [ ]:
df_ap.loc[:, "l4"] = df_ap["l4"].fillna(df_ap["l3"])

In [ ]:
df_ap["l4"].head()

,l4
id,
1068,Palermo Viejo
1069,Palermo
1073,Floresta
1082,Villa Crespo
1091,Villa del Parque


In [ ]:
df_ap_index = df_ap.index
df_ap = df_ap.merge(gb, on=["property_type", "l4"], how="left")
df_ap.index = df_ap_index


In [ ]:
# 1. Guardar el índice original (sin resetear)
df_ap_index = df_ap.index

# 2. Crear una copia del DataFrame sin 'price_m2'
df_ap_tmp = df_ap.drop(columns="price_m2", errors="ignore").copy()

# 3. Agregar el índice como columna temporal solo en esta copia para el merge
df_ap_tmp["id"] = df_ap_index

# 4. Hacer el merge
df_ap_tmp = df_ap_tmp.merge(gb, on=["property_type", "l4"], how="left")

# 5. Restaurar el índice original
df_ap_tmp = df_ap_tmp.set_index("id")

# 6. Asignar el resultado al df_ap original
df_ap = df_ap_tmp

# 7. Calcular la columna nueva sin perder el índice
df_ap["price_m2_x_surface_total"] = df_ap["price_m2"] * df_ap["surface_total"]


In [ ]:
df_ap.loc[df_ap["currency"] == "ARS", "price"] = df_ap.loc[df_ap["currency"] == "ARS", "price"] / PESOS_POR_DOLAR

In [ ]:
df_ap.loc[:, "bedrooms"] = df_ap["bedrooms"].fillna(df_ap["rooms"] - 1)
df_ap.loc[:, "rooms"] = df_ap["rooms"].fillna(df_ap["bedrooms"] + 1)

In [ ]:
#BATHS DIFERENCIA PORQUE SE LE DIO LA GANA AL DOCENTE
df_ap.loc[:, "bathrooms"] = df_ap["bathrooms"].fillna(df_ap["rooms"] - df_ap["bedrooms"])

In [ ]:
df_ap.loc[df_ap['property_type'] == 'Cochera', ['bathrooms', 'bedrooms', 'rooms']] = 0

In [ ]:
df_ap["surface_total"] = df_ap["surface_total"].fillna(df_ap["surface_covered"])
df_ap = df_ap.loc[df_ap["surface_total"].notna()]

In [ ]:
df_ap["ratio_cubierta"] = df_ap["surface_covered"] / df_ap["surface_total"]
df_ap["diferencia_superficie"] = df_ap["surface_total"] - df_ap["surface_covered"]

In [ ]:
# 1. Guardamos las columnas dummy de df_ent
columnas_dummy_ent = [col for col in df_ent.columns if col.startswith("prop_") or col.startswith("l4_")]

# 2. Creamos las dummies para df_ap
df_ap_dummies = pd.get_dummies(df_ap[["property_type", "l4"]], prefix=["prop", "l4"])

# 3. Agregamos columnas faltantes (con ceros) para que coincida con df_ent
for col in columnas_dummy_ent:
    if col not in df_ap_dummies.columns:
        df_ap_dummies[col] = 0

# 4. Alineamos el orden de columnas
df_ap_dummies = df_ap_dummies[columnas_dummy_ent]

# 5. Concatenamos al df_ap original
df_ap = pd.concat([df_ap, df_ap_dummies], axis=1)


In [ ]:
df_ap['rooms_to_surface_total'] = np.where(df_ap['surface_total'] != 0, df_ap['rooms'] / df_ap['surface_total'], np.nan)
df_ap['bedrooms_to_rooms'] = np.where(df_ap['rooms'] != 0, df_ap['bedrooms'] / df_ap['rooms'], np.nan)
df_ap['surface_covered_to_surface_total'] = np.where(df_ap['surface_total'] != 0, df_ap['surface_covered'] / df_ap['surface_total'], np.nan)
df_ap['bathrooms_to_rooms'] = np.where(df_ap['rooms'] != 0, df_ap['bathrooms'] / df_ap['rooms'], np.nan)
df_ap['bathrooms_to_bedrooms'] = np.where(df_ap['bedrooms'] != 0, df_ap['bathrooms'] / df_ap['bedrooms'], np.nan)
df_ap['rooms_plus_bedrooms'] = df_ap['rooms'] + df_ap['bedrooms']
df_ap['rooms_minus_bedrooms'] = df_ap['rooms'] - df_ap['bedrooms']


In [ ]:
df_ap["a_estrenar"] = df_ap["description"].str.lower().str.contains("estrenar", na=False)

In [ ]:
df_ap= df_ap.drop([
    "ad_type", "l6", "title", "description", "start_date", "end_date", "created_on",
    "currency", "price_period", "operation_type", "l1", "l2", "l5"
], axis=1)

In [ ]:
df_ap.head()

,lat,lon,l3,l4,rooms,bedrooms,bathrooms,surface_total,surface_covered,property_type,...,l4_Villa Urquiza,l4_Villa del Parque,rooms_to_surface_total,bedrooms_to_rooms,surface_covered_to_surface_total,bathrooms_to_rooms,bathrooms_to_bedrooms,rooms_plus_bedrooms,rooms_minus_bedrooms,a_estrenar
id,,,,,,,,,,,,,,,,,,,,,
1068,NaN,NaN,Palermo,Palermo Viejo,4.0,3.0,1.0,286.0,160.0,Casa,...,False,False,0.013986,0.750000,0.559441,0.250000,0.333333,7.0,1.0,False
1069,-58.426576,-34.590987,Palermo,Palermo,8.0,7.0,5.0,444.0,270.0,Casa,...,False,False,0.018018,0.875000,0.608108,0.625000,0.714286,15.0,1.0,False
1073,-58.479808,-34.631266,Floresta,Floresta,4.0,3.0,2.0,166.0,120.0,Casa,...,False,False,0.024096,0.750000,0.722892,0.500000,0.666667,7.0,1.0,False
1082,-58.437889,-34.603291,Villa Crespo,Villa Crespo,5.0,4.0,3.0,352.0,185.0,Casa,...,False,False,0.014205,0.800000,0.525568,0.600000,0.750000,9.0,1.0,False
1091,-58.476461,-34.602494,Villa del Parque,Villa del Parque,3.0,2.0,2.0,168.0,106.0,Casa,...,False,True,0.017857,0.666667,0.630952,0.666667,1.000000,5.0,1.0,False


In [ ]:
X = df_ent[df_ent.columns.drop('price')]
y = df_ent['price']

# Entrenamos el modelo con todos los datos de entrenamiento.csv, con los mejores hiperparámetros
n_estimators = 50
max_depth = 50
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=10, min_samples_split=20, n_jobs=-1, random_state=42)
reg.fit(X, y)

RandomForestRegressor(max_depth=50, min_samples_leaf=10, min_samples_split=20,
                      n_estimators=50, n_jobs=-1, random_state=42)

In [ ]:
# Hacemos en df_ap la misma limpieza que en df_ent
df_ap = df_ap.fillna(0)

df_ap = df_ap.select_dtypes(['number','bool'])

X_ap = df_ap[X.columns]
# y_ap = df_ap['price'] # no es necesario!

# Predecimos los precios del dataset a predecir
y_pred_ap = reg.predict(X_ap)
y_pred_ap

array([475588.26046146, 800178.23091091, 249879.26942635, ...,
       792155.3466808 , 200446.18639355, 242313.73938742])

## 5. Importancia de las columnas

In [ ]:
X = df_ent[df_ent.columns.drop('price')]
y = df_ent['price']

# Entrenamos el modelo con todos los datos de entrenamiento.csv, con los mejores hiperparámetros
n_estimators = 50
max_depth = 50
reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=10, min_samples_split=20, n_jobs=-1, random_state=42)
_ = reg.fit(X, y)

In [ ]:
# Importancia de las columnas
pd.DataFrame({"importance": reg.feature_importances_}, index=X.columns).sort_values(by="importance", ascending=False)

,importance
price_m2_x_surface_total,0.919507
lat,0.014454
diferencia_superficie,0.010421
lon,0.008095
ratio_cubierta,0.007670
...,...
l4_Parque Chas,0.000000
l4_Velez Sarsfield,0.000000
l4_Villa Soldati,0.000000
l4_Villa Real,0.000000


## 6. Análisis de errores

In [ ]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

n_estimators = 50
max_depth = 50

reg = reg = sk.ensemble.RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=10, min_samples_split=20, n_jobs=-1, random_state=42)
_ = reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

In [ ]:
X_test["error"] = abs(y_pred - y_test)
X_test["price"] = y_test
X_test["pred_price"] = y_pred
X_test.sort_values(by="error", ascending=False)

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price_m2,price_m2_x_surface_total,ratio_cubierta,...,bedrooms_to_rooms,surface_covered_to_surface_total,bathrooms_to_rooms,bathrooms_to_bedrooms,rooms_plus_bedrooms,rooms_minus_bedrooms,a_estrenar,error,price,pred_price
49235,-58.430505,-34.605452,3.0,2.0,1.0,6740.0,5950.0,2612.936216,1.761119e+07,0.882789,...,0.666667,0.882789,0.333333,0.500000,5.0,1.0,False,2.030140e+06,1.061571e+05,2.136297e+06
72344,-58.446369,-34.567543,5.0,4.0,4.0,380.0,380.0,3438.008455,1.306443e+06,1.000000,...,0.800000,1.000000,0.800000,1.000000,9.0,1.0,False,1.475396e+06,1.974522e+04,1.495141e+06
5673,-58.446369,-34.567543,8.0,7.0,5.0,1200.0,1200.0,2900.394636,3.480474e+06,1.000000,...,0.875000,1.000000,0.625000,0.714286,15.0,1.0,False,1.455425e+06,3.600000e+06,2.144575e+06
54632,-58.447649,-34.550990,5.0,4.0,4.0,600.0,400.0,2900.394636,1.740237e+06,0.666667,...,0.800000,0.666667,0.800000,1.000000,9.0,1.0,False,1.426089e+06,1.910828e+04,1.445198e+06
48377,-58.400247,-34.579672,6.0,5.0,4.0,970.0,970.0,5798.119340,5.624176e+06,1.000000,...,0.833333,1.000000,0.666667,0.800000,11.0,1.0,False,8.983304e+05,4.600000e+06,3.701670e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64265,-58.436266,-34.577712,1.0,0.0,1.0,38.0,34.0,3445.971902,1.309469e+05,0.894737,...,0.000000,0.894737,1.000000,0.000000,1.0,1.0,True,6.209608e+00,1.300000e+05,1.300062e+05
54923,-58.402925,-34.604328,3.0,2.0,1.0,81.0,75.0,1932.463774,1.565296e+05,0.925926,...,0.666667,0.925926,0.333333,0.500000,5.0,1.0,False,3.561404e+00,1.698000e+05,1.697964e+05
72151,-58.392273,-34.593705,2.0,1.0,1.0,44.0,44.0,3339.269338,1.469279e+05,1.000000,...,0.500000,1.000000,0.500000,1.000000,3.0,1.0,False,3.303388e+00,1.420000e+05,1.420033e+05
50239,-58.427724,-34.596052,3.0,2.0,1.0,90.0,77.0,3445.971902,3.101375e+05,0.855556,...,0.666667,0.855556,0.333333,0.500000,5.0,1.0,False,2.938326e+00,2.514360e+05,2.514389e+05


In [ ]:
df_desc = X_test.sort_values(by="error", ascending=False)
df_desc.head(100).to_csv("mayores_errores.csv")
